# Hybrid Quantum-Classical Optimization

To really highlight the capabilities of PennyLane, let’s now combine the qubit-rotation example from `default.qubit`, CV Photon-redirection we implemented using `strawberryfields.fock` and some classical processing to produce a truly hybrid computational model.

In [1]:
# import the essentials
import pennylane as qml
from pennylane import numpy as np

First, we define a computation consisting of:

1. two quantum nodes (the qubit rotation and photon redirection circuits running on the `default.qubit` and `strawberryfields.fock` devices, respectively)
2. a classical function, that simply returns the squared difference of its two inputs using NumPy

In [2]:
# create the devices
dev_qubit = qml.device('default.qubit', wires=1)
dev_fock = qml.device('strawberryfields.fock', wires=2, cutoff_dim=10)

# Qubit rotation QNode
@qml.qnode(dev_qubit)
def qubit_rotation(phi1, phi2):
    qml.RX(phi1, wires=0)
    qml.RY(phi2, wires=0)
    return qml.expval.PauliZ(0)

# The photon redirection QNode
@qml.qnode(dev_fock)
def photon_redirection(params):
    qml.FockState(1, wires=0)
    qml.Beamsplitter(params[0], params[1], wires=[0, 1])
    return qml.expval.MeanPhoton(1)

# Classical node to compute the squared difference between two inputs
def squared_difference(x, y):
    return np.abs(x-y)**2

Now, we can define an objective function associated with the optimization, linking together our three subcomponents. Here, we wish to perform the following hybrid quantum-classical optimization:

1. The qubit-rotation circuit will contain **fixed** rotation angles $\phi_1$ and $\phi_2$

2. The photon-redirection circuit will contain two **free** parameters- the beamsplitter angles $\theta$ and $\phi$- which are to be optimized.

3. The outputs of both QNodes will then be fed into the classical node, returning the squared difference of the two quantum functions.

4. Finally, the optimizer will calculate the gradient of the entire computation with respect to the free parameters $\theta$ and $\phi$ and update their values.

In essence, we are optimizing the photon-redirection circuit to return the **same expectation value** as the qubit-rotation circuit, even though they are two completely independent quantum systems.

We can translate this computational graph to the following function which combines the three nodes into a single hybrid computation. Below, we choose default values $\phi_1=0.5$, $\phi_2=0.1$:

In [3]:
def cost(params, phi1=0.5, phi2=0.1):    
    qubit_result = qubit_rotation(phi1, phi2)
    photon_result = photon_redirection(params)
    return squared_difference(qubit_result, photon_result)

Now, we use the built-in `GradientDescentOptimizer` to perform the optimization for 100 steps. As before, we choose initial beamsplitter parameters of $\theta=0.01$, $\phi=0.01$:

In [4]:
# initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.4)

# set the number of steps
steps = 100
# set the initial parameter values
params = np.array([0.01, 0.01])

for i in range(steps):
    # update the circuit parameters
    params = opt.step(cost, params)

    if (i+1) % 5 == 0:
        print('Cost after step {:5d}: {: .7f}'.format(i+1, cost(params)))

Cost after step     5:  0.2154539
Cost after step    10:  0.0000982
Cost after step    15:  0.0000011
Cost after step    20:  0.0000000
Cost after step    25:  0.0000000
Cost after step    30:  0.0000000
Cost after step    35:  0.0000000
Cost after step    40:  0.0000000
Cost after step    45:  0.0000000
Cost after step    50:  0.0000000
Cost after step    55:  0.0000000
Cost after step    60:  0.0000000
Cost after step    65:  0.0000000
Cost after step    70:  0.0000000
Cost after step    75:  0.0000000
Cost after step    80:  0.0000000
Cost after step    85:  0.0000000
Cost after step    90:  0.0000000
Cost after step    95:  0.0000000
Cost after step   100:  0.0000000


In [5]:
print('Optimized rotation angles: {}'.format(params))

Optimized rotation angles: [1.20671364 0.01      ]


Indeed, substituting this into the photon redirection QNode shows that it now produces the same output as the qubit rotation QNode:

In [6]:
result = [1.20671364, 0.01]
photon_redirection(result)

0.8731983021146449

In [7]:
qubit_rotation(0.5, 0.1)

0.8731983044562817

This is just a simple example of the kind of hybrid computation that can be carried out in PennyLane. Quantum nodes (bound to different devices) and classical functions can be combined in many different and interesting ways.